In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

from torchvision.transforms import ToTensor

In [11]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, num_classes, smoothing=0.0):
        super(LabelSmoothingLoss, self).__init__()
        self.num_classes = num_classes
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing
        
    def forward(self, pred, target):
        one_hot = torch.zeros_like(pred).scatter(1, target.unsqueeze(1),1)
        smooth_label = one_hot * self.confidence + (1-one_hot)*self.smoothing / (self.num_classes - 1)
        loss = torch.sum(-smooth_label * torch.log_softmax(pred, dim=1), dim=1)
        
        return torch.mean(loss)

In [12]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*5*5, 128)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x

dataset load, preprocessing

In [13]:
train_dataset = torchvision.datasets.FashionMNIST(root="./data/Fashion/", train=True,transform= ToTensor(), download=True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 64, shuffle = True)

In [14]:
model = MyModel()
print(model)

MyModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (relu3): ReLU(inplace=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [15]:

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_classes = 10
smoothing = 0.2
criterion = LabelSmoothingLoss(num_classes,smoothing = 0.0)
criterion_smoothing = LabelSmoothingLoss(num_classes, smoothing)

In [16]:
num_epochs = 20
train_losses_arr = []
train_losses_smoothing_arr = []

print(criterion)

for epoch in range(num_epochs):
    train_losses = 0.0
    train_losses_smoothing = 0.0
    
    for imgs, labels in train_dataloader:
        optimizer.zero_grad()
        
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses += loss.item()
        
        
        outputs_smoothing = model(imgs)
        loss_smoothing = criterion_smoothing(outputs_smoothing, labels)
        loss_smoothing.backward()
        optimizer.step()
        train_losses_smoothing += loss_smoothing.item()
        
    train_losses_arr.append(train_losses/len(train_dataloader))
    train_losses_smoothing_arr.append(train_losses_smoothing/len(train_dataloader))



LabelSmoothingLoss()
